In [1]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy import linalg
#from scipy import optimize
import matplotlib as mpl
import copy
from collections import defaultdict
from sklearn_extra.cluster import KMedoids
import math
import csv
import matlab.engine

In [2]:
dataset_name = 'creditcard'
cluster_size = 2

In [3]:
centers_opt = np.load('datafiles/adversarial_centers/' + dataset_name + '_centers_' + str(cluster_size) + '.npy')

####################
tmp_cntrs = []
for center in centers_opt:
    tmp_cntrs.append(np.sum(center))
tmp_cntrs = np.array(tmp_cntrs)
argsort_tmp_cntrs = np.argsort(tmp_cntrs)
final_cntrs = []
for i,_ in enumerate(centers_opt):
    final_cntrs.append(centers_opt[argsort_tmp_cntrs[i]])
final_cntrs = np.array(final_cntrs)
centers_opt = final_cntrs
####################

In [4]:
pr_groups = []
with open('./datafiles/groups_'+ dataset_name + '.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        pr_groups.append(row)
pr_groups = np.array(pr_groups).astype(np.float)
pr_groups = pr_groups - 1
print(pr_groups.shape)

if dataset_name == 'adultm' or dataset_name == 'seeds' or dataset_name == 'bank' or dataset_name == 'bankm':
    pr_groups = pr_groups.reshape((pr_groups.shape[-1], 1))
print(pr_groups, pr_groups.shape)

(30000, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]] (30000, 1)


In [5]:
def fairness_cost_balance(data, lbls, groups):
    
    npoints = data.shape[0]
    num_groups = np.unique(groups).shape[0]

    ideal_proportion = defaultdict(float)
    for g in range(num_groups):
        ideal_proportion[g] = np.count_nonzero(np.array(groups == g))

    for g in range(num_groups):
        ideal_proportion[g] /= float(npoints)
    
    membership = defaultdict(lambda: defaultdict(float))
    cluster_sizes = defaultdict(float)
    for idx, x in enumerate(data):
        cluster_k = lbls[idx]
        for g in range(num_groups):
            if groups[idx] == g:
                membership[g][cluster_k] += 1.0
                 
        cluster_sizes[cluster_k] += 1.0

    val = float('inf')
    
    for cluster_k in np.unique(lbls):
        for g in range(num_groups):
            if(membership[g][cluster_k] == 0):
                return 0
            
            a = (float(membership[g][cluster_k])/float(cluster_sizes[cluster_k]))/float(ideal_proportion[g])
            b = float(ideal_proportion[g])/(float(membership[g][cluster_k])/float(cluster_sizes[cluster_k]))
            val = min(min(a, b), val)
    
    return val

In [6]:
plotting_on = 1

In [7]:
def run_kmedoids(n_clusters, X):
    #kmed = KMedoids(n_clusters = cluster_size, random_state=42).fit(X)
    eng = matlab.engine.start_matlab()
    matlab_X = matlab.double(X.tolist())
    #matlab_labels, matlab_centers = eng.kmedoids(matlab_X, cluster_size, 'Distance', 'Euclidean' ,nargout = 2)
    matlab_labels, matlab_centers = eng.kmedoids(matlab_X, cluster_size, nargout = 2)
    
    centers = []
    for cntr in matlab_centers:
        centers.append(np.array(cntr))
    centers = np.array(centers)
    
    
    tmp_cntrs = []
    for center in centers:
        tmp_cntrs.append(np.sum(center))
    tmp_cntrs = np.array(tmp_cntrs)
    argsort_tmp_cntrs = np.argsort(tmp_cntrs)
    final_cntrs = []
    for i,_ in enumerate(centers):
        final_cntrs.append(centers[argsort_tmp_cntrs[i]])
    final_cntrs = np.array(final_cntrs)
    centers = final_cntrs

    labels = tmp_kmeans(X, centers)
    
    return (labels, centers)

In [8]:
def plot_clusters(X, labels, centers):
    chosen_pts_cluster_label = []
    
    for idx, label in enumerate(labels):
        if idx in mask_idx:
            if plotting_on:
                plt.scatter(X[idx,0], X[idx,1], color = 'orange', marker = 's', s = 30)
                continue
        
        if label == 0:
            plt.scatter(X[idx,0], X[idx,1], color = 'lawngreen',s = 30)
            
        if label == 1:
            plt.scatter(X[idx,0], X[idx,1], color = 'red', s = 30)            
                
            
    if plotting_on:
        plt.scatter(centers[0,0], centers[0,1], color = 'blue', marker = '*', s = 30)
        
        plt.scatter(centers[1,0], centers[1,1], color = 'purple', marker = '*', s = 30)

    
    plt.show()

In [9]:
def tmp_kmeans(data, cntrs):
    tmp_labels = []
    for idx,x in enumerate(data):
        if cluster_size == 1:
            tmp_labels.append(0)
        if cluster_size == 2:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x)]))
        if cluster_size == 3:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x)]))
        if cluster_size == 4:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x), linalg.norm(cntrs[3] - x)]))
        if cluster_size == 5:
            tmp_labels.append(np.argmin([linalg.norm(cntrs[0] - x), linalg.norm(cntrs[1] - x), linalg.norm(cntrs[2] - x), linalg.norm(cntrs[3] - x), linalg.norm(cntrs[4] - x)]))
            
    
    return tmp_labels

In [10]:
mask_idx = []
X = []
with open('./datafiles/data_' + dataset_name + '.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        X.append(row)
X = np.array(X).astype(np.float)
print(X.shape)
sample_size = X.shape[0]

(30000, 23)


In [11]:
labels, centers = run_kmedoids(cluster_size, X)
print(centers)


[[ 0.18583   0.81015  -0.13476   0.85854  -0.16115   0.01486   0.11173
   0.13886   0.18874   0.23491   0.25313  -0.44261  -0.42639  -0.45111
  -0.4874   -0.43108  -0.4107   -0.16081  -0.12678  -0.21995   0.011102
  -0.11777  -0.012122]
 [ 0.18583   0.81015   0.86719   0.85854   0.1643    0.01486   0.11173
   0.13886   0.18874   0.23491   0.25313   2.2049    2.0802    2.1143
   1.8342    1.7855    1.9501    0.20143   0.17703   0.15757   0.043018
   0.24221   0.016003]]


In [12]:
print(fairness_cost_balance(X, labels, pr_groups))

0.8611658242781424


In [13]:
labels_opt = tmp_kmeans(X, centers_opt)
print(fairness_cost_balance(X, labels_opt, pr_groups))

1.0


In [14]:
# temp_centers_opt = []
# for c_opt in centers_opt:
#     mincval = 10000
#     minc = -1
#     mincidx = -1
#     for idx,x in enumerate(X):
#         if np.linalg.norm(c_opt - x) < mincval:
#             mincval = np.linalg.norm(c_opt - x)
#             minc = x
#     temp_centers_opt.append(minc)
    
# centers_opt = temp_centers_opt
# print(centers_opt)

In [15]:
print(linalg.norm(centers - centers_opt))

50.426004203830935


In [16]:
V_k = [0]*cluster_size
ep_k = [100]*cluster_size
while np.linalg.norm(centers - centers_opt) != 0:
    ######## This code segment checks to see if we have arrived at similar centers since they might be shuffled (clustering is never consistent in labeling) #########################
    if cluster_size >= 2:
        done = 0
        for a in range(cluster_size):
            for b in range(cluster_size):
                if np.linalg.norm(centers[a] - centers_opt[b]) == 0:
                    done += 1
                    continue
        if done == cluster_size:
            break
    #################################
    for k_i in range(cluster_size):
        if np.linalg.norm(centers[k_i] - centers_opt[k_i]) != 0:
            V = datasets.make_blobs(n_samples=ep_k[k_i], cluster_std=[0.0], random_state=170, centers = [centers_opt[k_i]])[0] #RS = 42000
            X = np.vstack((X, V))
            for i in range(sample_size, sample_size+ep_k[k_i]):
                mask_idx.append(i)
            sample_size += ep_k[k_i]
            V_k[k_i] += ep_k[k_i]
        labels, centers = run_kmedoids(cluster_size, X)
        if np.linalg.norm(centers - centers_opt) == 0:
            break

    print(np.linalg.norm(centers-centers_opt))

0.0


In [17]:
print(X.shape)

(30200, 23)


In [18]:
print(V_k)

[100, 100]


In [19]:
print(centers)
print(centers_opt)

[[ 1.85830000e-01  8.10150000e-01  2.50610000e-01  8.58540000e-01
  -1.61150000e-01  1.48600000e-02  1.11730000e-01  1.38860000e-01
   1.88740000e-01  2.34910000e-01  2.53130000e-01 -2.76910000e-01
  -2.08740000e-01 -1.48350000e-01 -1.01250000e-01 -2.85210000e-03
   6.02350000e-02 -1.00440000e-01 -1.26780000e-01 -1.26410000e-01
  -5.27300000e-02 -1.17770000e-01 -1.24620000e-01]
 [ 8.72021218e+00  1.18418514e+00  1.54682886e+01  7.55035142e+00
   8.95993128e+00 -8.67718156e+00  4.16623419e+00 -6.40324172e+00
  -1.31144910e+01  1.05312929e+01 -1.43802721e+01 -7.69526512e+00
   9.75516352e+00 -6.65460946e+00  1.38444926e+01  1.08533665e+01
   2.27211129e+01 -1.12573129e+01  1.06127802e+01  2.36656753e+00
   2.42978370e+00  1.46165820e+01 -1.23342114e+01]]
[[ 1.85830000e-01  8.10150000e-01  2.50610000e-01  8.58540000e-01
  -1.61150000e-01  1.48600000e-02  1.11730000e-01  1.38860000e-01
   1.88740000e-01  2.34910000e-01  2.53130000e-01 -2.76910000e-01
  -2.08740000e-01 -1.48350000e-01 -1.01

In [22]:
if dataset_name == 'bankm': #'sanity_check'
    labels = tmp_kmeans(X, centers)
    print(fairness_cost_balance(X[:45211], labels, pr_groups))
    
    labels, centers = run_kmedoids(cluster_size, X)
    print(fairness_cost_balance(X[:45211], labels, pr_groups))
    
    
if dataset_name == 'creditcard': #'sanity_check'
    labels = tmp_kmeans(X, centers)
    print(fairness_cost_balance(X[:30000], labels, pr_groups))
    
    labels, centers = run_kmedoids(cluster_size, X)
    print(fairness_cost_balance(X[:30000], labels, pr_groups))


0
0


In [21]:
#BANKM: 45211, #ADULTM: 10000, #CREDITCARD: 30000

## k = 2
#BANKM: 100% decrease (100 [100, 0] points added) | 0.30374129684872203 -> 0.0
#CREDITCARD: 100% decrease (200 [100,100] points added) | 0.8611658242781424 -> 0.0
#ADULTM: 100% decrease (600 [500, 100] points added) | 0.5668357597268627 -> 0.0

## k = 3
#BANKM: 100% decrease (1300 [100, 600, 600] points added) | 0.29779400214116925 -> 0.0
#CREDITCARD: 100% decrease (300 [100, 100, 100] points added) | 0.7579775571824116 -> 0.0
#ADULTM: NA | 0.0 -> 0.0

## k = 4
#BANKM: 100% decrease (1900 [100, 200, 700, 900] points added) | 0.28292193008885463 -> 0.0
#CREDITCARD: 100% decrease (900 [400, 300, 100, 100] points added) | 0.4640690543560836 -> 0.0
#ADULTM: NA | 0.0 -> 0.0